In [1]:
!ls /kaggle/input/

fashion-dataset  fashion-product-images-dataset


In [2]:
!pip install pytorch-fid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.6 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [5]:
!cp -r /kaggle/input/fashion-dataset/val_images ./real_images
!cp -r /kaggle/input/fashion-dataset/results/generated_val_images ./fake_images

In [11]:
from PIL import Image
import os
from tqdm import tqdm
import time

def resize_folder(folder_path, size=(512, 512), save_folder=None):
    """
    Resize tất cả ảnh trong folder về kích thước size.
    - folder_path: đường dẫn folder ảnh gốc
    - size: tuple (width, height)
    - save_folder: nếu None → ghi đè ảnh gốc; nếu có → lưu vào folder mới
    """
    if save_folder:
        os.makedirs(save_folder, exist_ok=True)

    files = os.listdir(folder_path)
    start_time = time.time()

    for filename in tqdm(files, desc=f"Resizing {folder_path}", unit="img"):
        filepath = os.path.join(folder_path, filename)
        try:
            img = Image.open(filepath).convert("RGB")  # convert RGB
            img = img.resize(size, Image.Resampling.LANCZOS)
            
            if save_folder:
                save_path = os.path.join(save_folder, filename)
            else:
                save_path = filepath
            
            img.save(save_path)
        except Exception as e:
            print(f"Không xử lý được {filename}: {e}")

    elapsed = time.time() - start_time
    print(f"Đã resize {len(files)} ảnh trong {elapsed:.2f} giây.")

resize_folder("./real_images", size=(512, 512))


Resizing ./real_images: 100%|██████████| 4479/4479 [03:15<00:00, 22.94img/s] 

Đã resize 4479 ảnh trong 195.25 giây.


In [12]:
!python -m pytorch_fid ./real_images ./fake_images --device cuda:0 --num-workers 0 

100%|███████████████████████████████████████████| 85/85 [01:21<00:00,  1.04it/s]
FID:  47.21879198054836


In [25]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ny974f5q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ny974f5q
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=11c1362400da3b642b58ff4ad630bec895685db37ac08c87813f10f05637bb3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-rypbbfrg/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [27]:
!pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.7 MB/s eta 0:00:0000:0100:01


In [31]:
import torch
import open_clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
tokenizer = open_clip.get_tokenizer('ViT-B-32')
model.to(device).eval()

def clip_score_single_openclip(image_path, caption):
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
    text = tokenizer([caption]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        score = (image_features @ text_features.T).item()
        return score


In [36]:
import os
import pandas as pd
from tqdm import tqdm

folder_path = "/kaggle/input/fashion-dataset/results/generated_val_images"
csv_val = "/kaggle/input/fashion-dataset/val_fixed.csv"

image_files = sorted([f for f in os.listdir(folder_path) 
                      if f.lower().endswith((".png", ".jpg", ".jpeg"))])
df = pd.read_csv(csv_val)
captions = df["caption"]
val_captions = captions[:len(image_files)]
clip_score_mean = 0
for i in tqdm(range(len(val_captions)), desc="Calculating CLIP score"):
    image_path = os.path.join(folder_path, image_files[i])
    clip_score_mean += clip_score_single_openclip(image_path, val_captions[i])

clip_score_mean /= len(val_captions)
print(f"CLIP_score = {clip_score_mean}")


Calculating CLIP score: 100%|██████████| 4244/4244 [01:58<00:00, 35.75it/s]

CLIP_score = 0.3195547363479445
